In [ ]:
%load_ext rich
%load_ext autoreload
%autoreload 2

The rich extension is already loaded. To reload it, use:
  %reload_ext rich
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from pathlib import Path
import polars as pl
import pointblank as pb
import numpy as np
from rich import print as rprint

from odyssey.core import *
from odyssey.explore import *

In [ ]:
from config.paths import RAW_DATA

In [ ]:
g220 = Dataset("G220_Q.sav", RAW_DATA)
lf, meta = g220.load_data()

In [ ]:
cols = [
    "ID",
    "G220_IPAQ_MOD_D", "G220_IPAQ_MOD_HPD", "G220_IPAQ_MOD_MPD", "G220_IPAQ_MOD_W", 
    "G220_IPAQ_VIG_D", "G220_IPAQ_VIG_HPD", "G220_IPAQ_VIG_MPD", "G220_IPAQ_VIG_W", 
    "G220_IPAQ_WALK_D", "G220_IPAQ_WALK_HPD", "G220_IPAQ_WALK_MPD", "G220_IPAQ_WALK_W", 
    "G220_IPAQ_SIT_WD_HPD", "G220_IPAQ_SIT_WD_MPD", 
    "G220_SIT_WD_TRUNC", "G220_IPAQ_SIT_COM",
    "G220_VIG_MET", "G220_VIG_MINS", 
    "G220_MOD_MET", "G220_MOD_MINS", 
    "G220_WALK_MET", "G220_WALK_MINS",
    "G220_IPAQ_CAT", "G220_TOT_MET", 
]

In [ ]:
df = lf.select(cols).collect()

Start building function factories to make functions more reusable.
Refactor to not only accept different exercise categories, but different generations.

In [ ]:
def check_total_mins(df: pl.DataFrame) -> pl.DataFrame:
    return df.with_columns(
        (pl.col("G220_IPAQ_VIG_HPD") * 60 + pl.col("G220_IPAQ_VIG_MPD"))
        # Cap values > 180 and preserve nulls
        .pipe(lambda expr: pl.when(expr > 180).then(180).otherwise(expr))
        .alias("check")
    )

def check_met(df: pl.DataFrame) -> pl.DataFrame:
    return df.with_columns(
        (pl.col("G220_VIG_MINS").fill_null(0) * pl.col("G220_IPAQ_VIG_D").fill_null(0) * 8).alias("check"),
        pl.col("G220_VIG_MET").fill_null(0)
    )

def check_tot_met(df: pl.DataFrame) -> pl.DataFrame:
    cols_to_sum = ["G220_VIG_MET", "G220_MOD_MET", "G220_WALK_MET"]
    expr = sum(pl.col(col).fill_null(0) for col in cols_to_sum)
    
    return df.with_columns(
        expr.alias("check"),
        pl.col("G220_TOT_MET").fill_null(0) # Fill nulls with 0; otherwise the validation skips if one value in a comparison is null
    )

In [ ]:
validation = (
    pb.Validate(
        data=df,
        brief="Step {step}: {auto}"
    )
    # If `VIG_W` was answered (0 or 1), corresponding values for `VIG_MINS` and `VIG_MET` exist are >= 0
    .col_vals_between(
        columns=pb.matches("VIG_MINS"), 
        left=0,
        right=180,
        segments=("G220_IPAQ_VIG_W", [0, 1]) # Only look at instances where there's the participant answered the question (ignore if skipped)
    )
    .col_vals_ge(
        columns=pb.matches("VIG_MET"), 
        value=0,
        segments=("G220_IPAQ_VIG_W", [0, 1]) # Only look at instances where there's the participant answered the question (ignore if skipped)
    )
    # If `VIG_W` is null, corresponding values for `VIG_MINS` and `VIG_MET` are also null
    # TODO: simplify so it just accepts and segments by null values, rather than filling null with -1
    .col_vals_null(
        columns=pb.matches("VIG_(MINS|MET)"),
        pre=lambda df: df.with_columns(pl.col("G220_IPAQ_VIG_W").fill_null(-1)), # Pointblank doesn't seem to like segmenting values with null, so transform null to -1 and segment y that
        segments=("G220_IPAQ_VIG_W", -1)
    )
    .col_vals_eq(
        columns=pb.matches("VIG_MET"), 
        value=0,
        segments=("G220_IPAQ_VIG_D", [0]) # Only look at instances where there's the participant answered the question (ignore if skipped)
    )
    .col_vals_eq(
        columns="G220_VIG_MINS",
        value=pb.col("check"), # the new created column
        pre=check_total_mins,
        brief="Check total mins/day equals `HPD*60 + MPD`"
    )
    .col_vals_eq(
        columns="G220_VIG_MET",
        value=pb.col("check"),
        pre=check_met,
    )
    .col_vals_eq(
        columns="G220_TOT_MET",
        value=pb.col("check"),
        pre=check_tot_met,
    )
    .interrogate()
)

validation

Validate(
    data=shape: (1_487, 25)
┌─────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ ID      ┆ G220_IPAQ_ ┆ G220_IPAQ ┆ G220_IPAQ ┆ … ┆ G220_WALK ┆ G220_WALK ┆ G220_IPAQ ┆ G220_TOT_ │
│ ---     ┆ MOD_D      ┆ _MOD_HPD  ┆ _MOD_MPD  ┆   ┆ _MET      ┆ _MINS     ┆ _CAT      ┆ MET       │
│ f64     ┆ ---        ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│         ┆ f64        ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞═════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 10010.0 ┆ 0.0        ┆ 0.0       ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 0.0       │
│ 10020.0 ┆ null       ┆ null      ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ 10030.0 ┆ 0.0        ┆ 0.0       ┆ 0.0       ┆ … ┆ null      ┆ 180.0     ┆ null      ┆ null      │
│ 10040.0 ┆ 1.0        ┆ 0.0       ┆ 30.0      ┆ … ┆ 4158.0    ┆ 180.0     ┆ 2.0       ┆ 5238.0    │
│ 10050.0 ┆ 4.0        ┆ 3.0       ┆ 0.0       ┆ … ┆ 231.0     ┆ 10.0      ┆ 2.0       ┆ 3591.0    │
│ …       ┆ …          ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …         │
│ 54540.0 ┆ null       ┆ null      ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ 54550.0 ┆ null       ┆ null      ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ 54580.0 ┆ 3.0        ┆ 4.0       ┆ 0.0       ┆ … ┆ 1584.0    ┆ 120.0     ┆ 2.0       ┆ 3744.0    │
│ 54590.0 ┆ 2.0        ┆ 0.0       ┆ 45.0      ┆ … ┆ 297.0     ┆ 30.0      ┆ 1.0       ┆ 657.0     │
│ 54602.0 ┆ 2.0        ┆ 0.0       ┆ 30.0      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
└─────────┴────────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴───────────┘,
    tbl_name=None,
    label=None,
    thresholds=Thresholds(warning=None, error=None, critical=None),
    actions=None,
    final_actions=None,
    brief='Step {step}: {auto}',
    lang='en',
    locale='en'
)